In [6]:
!pip install binance-connector

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.3 MB/s eta 0:00:00


In [9]:
from binance.spot import Spot
from datetime import datetime,timedelta
client = Spot()

In [10]:
from sagemaker.feature_store.feature_group import FeatureGroup
import boto3
import pandas as pd
import numpy as np
import io
from sagemaker.session import Session
from sagemaker import get_execution_role

# customers_feature_group_name = 'BtcPriceData' 
# customers_feature_group = FeatureGroup(
#     name=customers_feature_group_name, sagemaker_session=sagemaker_session
#)

feature_group = FeatureGroup(name = 'BtcPriceData')


In [11]:
def get_ts_for_days():
    start_date = datetime(2024, 1, 8)
    end_date = datetime(2024, 1, 11)
    day_count = (end_date - start_date).days + 1
    for  n in range(day_count):
        temp_date = start_date + timedelta(days = n)
        print('date',temp_date)
        yield temp_date, int(datetime.timestamp(temp_date)*1000)


In [12]:
for date, timestamp in get_ts_for_days():
    klines = client.klines("BTCUSDT", "5m",limit = 11*12  ,startTime = timestamp) #endTime = end_ts )#,limit = 1)#24 * 4)
    df = pd.DataFrame(klines, columns = ['open_time','open_price','high_price','low_price','close_price','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
    #df['open_time'] = pd.to_datetime(df['open_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
    #df['close_time'] = pd.to_datetime(df['close_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
    #df['EventTime'] = round(time.time())
    df['close_time'] =  df['close_time'].div(1000)
    df = df.drop(columns = ['ignore','taker_buy_quote_asset_volume'])
    df = df.astype(float)
    #print(df)
    #feature_group.load_feature_definitions(df)
    feature_group.ingest(data_frame=df, max_workers=2, wait=True)


date 2024-01-08 00:00:00
date 2024-01-09 00:00:00
date 2024-01-10 00:00:00
date 2024-01-11 00:00:00


In [ ]:
def get_ts_for_days():
    start_date = datetime(2024, 1, 3)
    end_date = datetime(2024, 1, 7)
    day_count = (end_date - start_date).days + 1
    for  n in range(day_count):
        temp_date = start_date + timedelta(days = n)
        print('date',temp_date)
        yield temp_date, int(datetime.timestamp(temp_date)*1000)


In [188]:
now = datetime.fromtimestamp(round(time.time()))
now.hour

19

In [192]:
now.time()

datetime.time(19, 23, 33)

In [ ]:
klines = client.klines("BTCUSDT", "5m",limit = 4*12  ,startTime = timestamp) #endTime = end_ts )#,limit = 1)#24 * 4)
df = pd.DataFrame(klines, columns = ['open_time','open_price','high_price','low_price','close_price','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
#df['open_time'] = pd.to_datetime(df['open_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
#df['close_time'] = pd.to_datetime(df['close_time'], unit='ms').dt.strftime('%Y-%m-%d %T')
#df['EventTime'] = round(time.time())
df['close_time'] =  df['close_time'].div(1000)
df = df.drop(columns = ['ignore','taker_buy_quote_asset_volume'])
df = df.astype(float)
#print(df)
#feature_group.load_feature_definitions(df)
feature_group.ingest(data_frame=df, max_workers=2, wait=True)